In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence

c:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\LSTM


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
torch.__version__

'2.0.1+cu117'

In [5]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [6]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [7]:
tracks = ['Sandown Park',
 'The Meadows',
 'Warragul',
 'Geelong',
 'Sale',
 'Ballarat',
 'Bendigo',
 'Cranbourne']

In [8]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ"]
data_file = 'gru_inputs_simple_kitchen_sink.fth'
raceDB = build_dataset(data_file, hidden_size ,state_filter=states, margin_type='boosted_sftmin',v6=True)
raceDB.create_new_weights_v2()

[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]
(1377299, 24)
26
Latest date = 2023-09-19 00:00:00
size after state filter (144207, 26)
(144207, 26)
Latest date = 2023-09-17 00:00:00
num_features_per_dog=26


  0%|          | 0/3841 [00:00<?, ?it/s]


KeyError: 'StartPrice'

In [ ]:
raceDB.create_test_split_date('2023-04-30')

Train examples 17160, Test examples 856, Val examples 1135


In [ ]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 32452, Test examples 11947


In [ ]:
raceDB.create_dogs_test_split_date()

100%|██████████| 15826/15826 [00:00<00:00, 61048.93it/s]


In [ ]:
raceDB.attach_races_to_dog_inputv2()

100%|██████████| 15827/15827 [00:03<00:00, 4241.97it/s]


In [ ]:
raceDB.get_dog_test([1,2])[0][0].stats

tensor([ 3.0000, 29.8000,  5.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000,  0.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000], device='cuda:0', dtype=torch.float64)

In [ ]:
raceDB.create_batches(batch_days=180)

[(datetime.date(2019, 12, 1), datetime.date(2020, 5, 29)), (datetime.date(2020, 5, 29), datetime.date(2020, 11, 25)), (datetime.date(2020, 11, 25), datetime.date(2021, 5, 24)), (datetime.date(2021, 5, 24), datetime.date(2021, 11, 20)), (datetime.date(2021, 11, 20), datetime.date(2022, 5, 19)), (datetime.date(2022, 5, 19), datetime.date(2022, 7, 31))]
2020-05-29
2020-11-25
2021-05-24
2021-11-20
2022-05-19
2022-07-31
Train examples [5484, 5851, 5780, 6424, 6281, 2594]


100%|██████████| 6/6 [00:59<00:00,  9.88s/it]


Train examples [4704, 4981, 4961, 5322, 5486, 4058]
Train examples [4704, 4981, 4961, 5322, 5486, 4058]
Train examples [5484, 5851, 5780, 6424, 6281, 2594]
Train examples [5484, 5851, 5780, 6424, 6281, 2594]


In [ ]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU", config=config, save_code=True):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)
      
      if 'batch_days' in config.keys():
        raceDB.create_test_split_date(config['training_date_end'])
        raceDB.create_dogs_test_split_date()
        raceDB.attach_races_to_dog_inputv2() 
        raceDB.reset_hidden()
        raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'])

      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          input_size = raceDB.batches['packed_x'][0].data[0].shape[0]


      print(f"{input_size=}")

      


      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_BN(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        print(model_data['model_state_dict'].keys())
        del_keys = ['gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0','layer_norm.weight', 'layer_norm.bias']
        [model_data['model_state_dict'].pop(k) for k in del_keys]


        model.load_state_dict(model_data['model_state_dict'], strict=False)
        config['parent model'] = prev_model_file
        model = model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
        # optimizer.load_state_dict(model_data['optim'])
        # optimizer.to(device)
      else:
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()

      criterion = nn.CrossEntropyLoss(reduction='none')

      model = model.to(device)
      print(model)

      # and use them to train the model
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru.train_regular_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      # dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")
      if sweep:
        # raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [ ]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':128,
                        'stats':raceDB.stats_cols,
                        'races':states,
                        'datafile':data_file,
                        'latest_date':raceDB.latest_date,
                        'input_type':'basic',
                        'num_layers':2,
                        'batch_size': 750,
                        'dropout': 0.3,
                        'epochs': 20,
                        'learning_rate': 0.0001,
                        'optimizer': 'adamW',
                        'f0_layer_size':128,
                        'f1_layer_size':64,
                        'training_date_end':'2023-04-30',
                        'batch_days':365}

In [ ]:
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [ ]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [ ]:
torch.cuda.empty_cache()

In [ ]:
for race in raceDB.racesDict.values():
    for dog in race.dogs:
        try:
            del dog.stats
        except:
            pass

torch.cuda.empty_cache()

In [ ]:
dogs = raceDB.get_dog_train((1,2))

In [ ]:
dogs[0][0].stats

AttributeError: 'DogInput' object has no attribute 'stats'

In [ ]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


{'hidden_size': 128, 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]", 'races': ['NZ'], 'datafile': 'gru_inputs_simple_kitchen_sink.fth', 'latest_date': '2023-09-17T00:00:00', 'input_type': 'basic', 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 20, 'learning_rate': 0.0001, 'optimizer': 'adamW', 'f0_layer_size': 128, 'f1_layer_size': 64, 'training_date_end': '2023-04-30', 'batch_days': 365}
20
{'hidden_size': 128, 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_

  0%|          | 0/3841 [00:00<?, ?it/s]

  0%|          | 0/3842 [00:00<?, ?it/s]

AttributeError: 'DogInput' object has no attribute 'stats'

In [ ]:
sweep_config = {"method": "grid"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "batch_days": {"values": [90,180,365,550,10000]},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "input_type": {"values": ['basic']},
    "num_layers": {"values": [2]},
    'hidden_size':{'values':[128]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "stats":{"value": raceDB.stats_cols},
    "races":{"value": states},
    "batch_size":{"value": 10},
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [500]},
        "validation_split": {"value": 0.1},
        "training_date_end": {"values": 
                                [
                                '2023-04-30',
                                '2023-02-28',
                                '2022-11-30',
                                '2021-11-30'
                                ]},
        "learning_rate": {"values": [0.0003, 0.0001]},
        "label_smoothing": {"values": [0.01,0.0]},
        "loss": {
            "values": [ "CEL"],
        },
    }
)
# )

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'batch_days': {'values': [90, 180, 365, 550, 10000]},
                'batch_size': {'value': 10},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [500]},
                'f0_layer_size': {'values': [128]},
                'f1_layer_size': {'values': [64]},
                'hidden_size': {'values': [128]},
                'input_type': {'values': ['basic']},
                'label_smoothing': {'values': [0.01, 0.0]},
                'learning_rate': {'values': [0.0001, 0.008, 0.02]},
                'len_data': {'value': 43651},
                'loss': {'values': ['CEL']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'races': {'value': ['NSW']},
                'stats': {'value': "[['dist_last__1', 'box_last__1', "
                                   "'speed_avg_1', 'split_speed_v1_1', "
            

{'method': 'grid',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'batch_days': {'values': [90, 180, 365, 550, 10000]},
  'f0_layer_size': {'values': [128]},
  'f1_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'input_type': {'values': ['basic']},
  'num_layers': {'values': [2]},
  'hidden_size': {'values': [128]},
  'len_data': {'value': 43651},
  'stats': {'value': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]"},
  'races': {'value': ['NSW']},
  'batch_size': {'value': 10},
  'epochs': {'values': [500]},
  'validation_split': {'value': 0.1},
  'learning_rate': {'values': [0.000

In [ ]:
wandb.agent('x9mz21wl', function=model_pipeline, count=100, project='NEW GRU-sweeps')

wandb: Agent Starting Run: xsj0dnt1 with config:
wandb: 	batch_days: 365
wandb: 	batch_size: 10
wandb: 	dropout: 0.3
wandb: 	epochs: 500
wandb: 	f0_layer_size: 128
wandb: 	f1_layer_size: 64
wandb: 	hidden_size: 128
wandb: 	input_type: basic
wandb: 	label_smoothing: 0
wandb: 	learning_rate: 0.0003
wandb: 	len_data: 52237
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	races: ['VIC']
wandb: 	stats: [['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignor

{'batch_days': 365, 'batch_size': 10, 'dropout': 0.3, 'epochs': 500, 'f0_layer_size': 128, 'f1_layer_size': 64, 'hidden_size': 128, 'input_type': 'basic', 'label_smoothing': 0, 'learning_rate': 0.0003, 'len_data': 52237, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'races': ['VIC'], 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]", 'validation_split': 0.1}
500
{'batch_days': 365, 'batch_size': 10, 'dropout': 0.3, 'epochs': 500, 'f0_layer_size': 128, 'f1_layer_size': 64, 'hidden_size': 128, 'input_type': 'basic', 'label_smoothing': 0, 'learning_rate': 0.0003, 'len_data': 52237, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'races'

  0%|          | 0/500 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp6jhpx56zwandb-media\\slpu6963.table.json'
created path


  4%|▍         | 20/500 [07:52<3:08:54, 23.61s/it]

...

 84%|████████▍ | 420/500 [2:43:24<31:47, 23.85s/it]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp6jhpx56zwandb-media\\m2l8j9z3.table.json'


 88%|████████▊ | 440/500 [2:51:19<23:16, 23.27s/it]

[Errno 2] No such file or directory: 'C:\\Users\\Nick\\AppData\\Local\\Temp\\tmp6jhpx56zwandb-media\\1kloe1kd.table.json'


 91%|█████████ | 456/500 [2:57:47<17:10, 23.41s/it]

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="NEW GRU-sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: ohses7v7
Sweep URL: https://wandb.ai/nickojelly/NEW%20GRU-sweeps/sweeps/ohses7v7


wandb: Agent Starting Run: ek2pxt8u with config:
wandb: 	batch_days: 90
wandb: 	batch_size: 10
wandb: 	dropout: 0.3
wandb: 	epochs: 500
wandb: 	f0_layer_size: 128
wandb: 	f1_layer_size: 64
wandb: 	hidden_size: 128
wandb: 	input_type: basic
wandb: 	label_smoothing: 0.01
wandb: 	learning_rate: 0.0001
wandb: 	len_data: 43651
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	races: ['NSW']
wandb: 	stats: [['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently l

{'batch_days': 90, 'batch_size': 10, 'dropout': 0.3, 'epochs': 500, 'f0_layer_size': 128, 'f1_layer_size': 64, 'hidden_size': 128, 'input_type': 'basic', 'label_smoothing': 0.01, 'learning_rate': 0.0001, 'len_data': 43651, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'races': ['NSW'], 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]", 'validation_split': 0.1}
500
{'batch_days': 90, 'batch_size': 10, 'dropout': 0.3, 'epochs': 500, 'f0_layer_size': 128, 'f1_layer_size': 64, 'hidden_size': 128, 'input_type': 'basic', 'label_smoothing': 0.01, 'learning_rate': 0.0001, 'len_data': 43651, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'ra

  0%|          | 0/500 [00:00<?, ?it/s]

...

wandb: Agent Starting Run: a58t4set with config:
wandb: 	batch_days: 90
wandb: 	batch_size: 10
wandb: 	dropout: 0.3
wandb: 	epochs: 500
wandb: 	f0_layer_size: 128
wandb: 	f1_layer_size: 64
wandb: 	hidden_size: 128
wandb: 	input_type: basic
wandb: 	label_smoothing: 0
wandb: 	learning_rate: 0.0001
wandb: 	len_data: 43651
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	races: ['NSW']
wandb: 	stats: [['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignore

{'batch_days': 90, 'batch_size': 10, 'dropout': 0.3, 'epochs': 500, 'f0_layer_size': 128, 'f1_layer_size': 64, 'hidden_size': 128, 'input_type': 'basic', 'label_smoothing': 0, 'learning_rate': 0.0001, 'len_data': 43651, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'races': ['NSW'], 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]", 'validation_split': 0.1}
500
{'batch_days': 90, 'batch_size': 10, 'dropout': 0.3, 'epochs': 500, 'f0_layer_size': 128, 'f1_layer_size': 64, 'hidden_size': 128, 'input_type': 'basic', 'label_smoothing': 0, 'learning_rate': 0.0001, 'len_data': 43651, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'races': 

  0%|          | 0/500 [00:00<?, ?it/s]

created path


 45%|████▌     | 226/500 [1:12:05<1:05:55, 14.44s/it]